In [6]:
# setup
from IPython.core.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('../rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})


# CMPS 2200
# Introduction to Algorithms

## Divide-and-Conquer & Contraction


## Review -  Reductions
 
[**Reduction**]: A problem $A$ is reducible to a problem $B$ if ``any`` instance of problem $A$ can be turned into some instance of $B$.

<span style="color:red">Comparison:</span> Problem, Algorithm, Instance?



<br><p>
<img width=70% src="reduction.jpg"/>
<br><p>
   
<span style="color:red">Question:</span> 
- Problem A: `median finding`
- Problem B: `finding the k-th smallest number in a list`
- Problem C: `sorting`


<span style="color:blue">Solution:</span> 
- `median finding` is reducible to `find the k-th smallest number in a list`
    
- `find the k-th smallest number in a list` is reducible to `Sorting`.




    
<span style="color:green">Lower Bound and Upper Bound</span>
  - $A$ is reducible to $B$, $B$ is reducible to $C$.

<span style="color:red">Brute Force</span> paradigm just for a problem $A$ and an instance $\mathcal{I}_A$, just looks at every possible solution [**search space**] and checks each one.


<span style="color:blue">Question:</span> Consider the problem of finding the maximum element in a list $L$ of length $n$. 


- What is the search space for this problem?


- What is the work/span of the brute-force algorithm? Please compare with `iterate` and `reduce`.




### Divide-and-Conquer

We've seen a few divide-and-conquer algorithms already, so let's look at the high-level approach. 


For a problem $A$ and instance $\mathcal{I}_A$:

- **Base Case**: If $\mathcal{I}_A$ is small, solve directly. 

- **Inductive Step**: 
    - **Divide** $\mathcal{I}_A$ into smaller instances.
    - **Recursively solve** smaller instances.
    - **Combine** solutions

<img width=70% src="dc_fig.jpeg"/>

<span style="color:blue">Efficient Reduction:</span> When the reduction is not efficient in terms of the three scenarios of `Brick Method`?
    


As we'll see, each algorithmic paradigm has high-level strategies to i) *prove correctness* and ii) *determine work/span*.

How do we prove the correctness of a divide-and-conquer algorithm?


```python

def Fibonacci(n):
    if n <= 1:
		return n
	else:
		return Fibonacci(n-1) + Fibonacci(n-2)
```

<span style="color:red">**Induction**</span> -- why and how?

Induction provides a natural framework for divide-and-conquer algorithms.

The **base case** of the induction requires us to prove that the algorithm works for the base case.

For the **induction step**, we use the inductive hypothesis that the solutions to the smaller instances are correct. Then, we must prove that the combine step correctly produces the desired solution. 

What about determining work/span?

We've seen that recurrences can capture the behavior of divide-and-conquer algorithms - they simply capture the cost of recursively solving smaller instances and then combining the solutions. 

The general form of the work is:

$$ W(n) = W_{\mathsf{\small divide}}(n) + \sum_{i=1}^{k}W(n_i) + W_{\mathsf{\small combine}}(n) + 1 $$

The general form for the span is:

$$ S(n) = S_{\mathsf{\small divide}}(n) + \max_{i=1}^{k}S(n_i) + S_{\mathsf{\small combine}}(n) + 1 $$



Let's look at how Merge Sort fits into this framework.


<p><span class="math display">\[\begin{array}{l}  
\mathit{mergeSort}~a =  
\\   
~~~~\texttt{if}~|a| \leq 1~\texttt{then}  
\\   
~~~~~~~~a  
\\  
~~~~\texttt{else}  
\\   
~~~~~~~~\texttt{let}  
\\  
~~~~~~~~~~~~(l,r) = \mathit{splitMid}~a  
\\   
~~~~~~~~~~~~(l',r') = (\mathit{mergeSort}~l \mid\mid{} \mathit{mergeSort}~r)  
\\  
~~~~~~~~\texttt{in}  
\\   
~~~~~~~~~~~~\mathit{merge} (l',r')  
\\  
~~~~~~~~\texttt{end}  
\end{array}\]</span></p>




For the correctness, we need to show that Merge Sort truly sorts the list. Let's perform induction:

- **Base case**: We correctly sort a singleton list.

- **Induction Step**: We assume that we can correctly sort the two halves of the list (by the inductive hypothesis). The final step is to prove that the merge step works correctly to combine two sorted lists into one sorted list (which we've shown previously). 


For the running time, recall that we characterized the work/span of Merge Sort as:

$$ W(n) = 2W(n/2) + O(n) $$
 
and

$$ S(n) = S(n/2) + O(\log n) $$
 
This fits into the framework above: the divide step takes $O(1)$ time, there are 2 concurrent recursive calls, and merging takes $O(n)$ work and $O(\log n)$ span.    

Recall that we gave a divide-and-conquer algorithm for `reduce`:

$reduce \: f \: id \: a =
\begin{cases}
id & \hbox{if} \: |a| = 0\\
a[0] & \hbox{if} \: |a| = 1\\
f(reduce \: f \: id \: (a[0 \ldots \lfloor \frac{|a|}{2} \rfloor - 1]), \\ \:\:\:reduce \: f \: id \: (a[\lfloor \frac{|a|}{2} \rfloor \ldots |a|-1])& \hbox{otherwise}
\end{cases}
$

`reduce(merge, [], list(map(singleton, [1,3,6,4,8,7,5,2])))`

This is Merge Sort! 

Can all divide-and-conquer algorithms be implemented with `reduce`?


The divide-and-conquer framework is much more general than `reduce`. So `reduce` cannot be used when, for example, we wish to split the input into 3 or more parts, or if they are of unequal size. 

### Contraction
A contraction algorithm for problem $A$ has the following structure.

- **Base Case**: If the problem instance is sufficiently small, then compute and return the solution, possibly using another algorithm.

- **Inductive Step(s)**: If the problem instance is sufficiently large, then 
  - Apply the following two steps, as many times as needed:

    - ``Contract``: map the instance of the problem $A$ to a smaller instance of $A$.
    - ``Solve``: solve the smaller instance recursively.

  - Expand the solutions to smaller instance to solve the original instance.
  
\begin{array}{l}  
\\  
\mathit{scan}~f~\mathit{id}~a =  
\\  
~~~~\texttt{if}~|a| = 0~\texttt{then}   
\\  
~~~~\left(\left\langle\,  \,\right\rangle, id\right)  
\\  
~~~~\texttt{else if}~|a| = 1 ~\texttt{then}  
\\  
~~~~~~~~\left( \left\langle\, id \,\right\rangle, a[0] \right)  
\\  
~~~~\texttt{else}  
\\   
~~~~~~~~\texttt{let}  
\\  
~~~~~~~~~~~~a' = \left\langle\,  f(a[2i],a[2i+1]) : 0 \leq i < n/2 \,\right\rangle  
\\  
~~~~~~~~~~~~(r,t) = \mathit{scan}~f~\mathit{id}~ a' 
\\  
~~~~~~~~\texttt{in}  
\\   
~~~~~~~~~~~~(\left\langle\,  p_i : 0 \leq i < n  \,\right\rangle, t),~\texttt{where}~p_i =   
\begin{cases}  
     r[i/2]  & \texttt{even}(i) \\  
     f(r[i/2], a[i-1]) & \texttt{otherwise}  
\end{cases}  
\\  
~~~~~~~~\texttt{end}  
\end{array}
  


  
### Remark: 
Contraction differs from divide and conquer in that it allows there to be only one independent smaller instance to be recursively solved. There could be multiple dependent smaller instances to be solved one after another (sequentially).

### Example (Maximal Element)

We can find the maximal element in a sequence $a$ using contraction as follows. If the sequence has only one element, we return that element, otherwise, we can map the sequence $a$ into a sequence $b$ which is half the length by comparing the elements of $a$ at consecutive even-odd positions and writing the larger into $b$. We then find the largest in $b$ and return this as the result.

For example, we map the sequence ⟨1,2,4,3,6,5⟩ to ⟨2,4,6⟩. The largest element of this sequence, 6 is then the largest element in the input sequence.

<br>
<br>
For a sequence of length $n$, we can write the work and span for this algorithm as recurrences as follows:

$$ \begin{equation}
W(n) = \begin{cases}
  \mathcal{O}(1), & \text{if $n=1$} \\
  W(n/2) + \mathcal{O}(n), & \text{otherwise} 
  \end{cases}
\end{equation}$$

$$ \begin{equation}
S(n) = \begin{cases}
  \Theta(1), & \text{if $n=1$} \\
  S(n/2) + \mathcal{O}(1), & \text{otherwise} 
  \end{cases}
\end{equation}$$

> We can solve the recurrences to obtain $W(n)=\mathcal{O}(n)$ and $S(n)=\mathcal{O}(\lg n)$.


<br>
<span style="color:red">Question:</span> What is the work/span for divide-and-conquer version? That is, `reduce(max, -$\infty$, a)`

<span style="color:red">Question:</span> How can we apply contraction in `Sorting`?

### Example: Euclidean Traveling Salesperson Problem

In the Euclidean Traveling Salesperson Problem (eTSP), you are given a set of $n$ 2D points. The goal is to find a "tour" that begins and ends with the same point such that:
- every point is visited exactly once (except the starting point) 
- the sum of distances between adjacent points is minimized

This is an incredibly widespread and useful problem -- consider all the various kinds of routing problems that are solved every day. For a simple example, think of Amazon/USPS/UPS package deliveries.

Which solution is better?

<br><p> 
 ![eTSP_simple.jpg](eTSP_simple.jpg)
<br><p> 

Given an input with $n$ points, how many possible solutions are there?

## Brute-Force?

If we take a brute-force approach to this problem, what is the solution space and how can we search it?

There are $n!$ possible solutions, and we must check the cost of each by summing $n$ distances. This can be done with $O(n)$ work and $O(\log n)$ span. So we can solve eTSP with $O(n\cdot n!)$ work and $O(\log n)$ span. 

<span style="color:red">Question:</span> Are we done if we use brute-force approach? 




## Divide-and-Conquer?

What intuition can we get about the fact that this problem is in 2D?


<br><p> 
 ![eTSP_harder_sol.jpg](eTSP_harder_sol.jpg)
<br><p> 

Since points that are "clustered" can possibly be dealt with first, how about a divide-and-conquer approach? How would that work?



We can split the input using a "cut" through the plane that separates the input points into two equal parts. Then, recursively solve eTSP for each smaller point set. 

How do we combine smaller solutions into larger ones?



We need to make sure that two tours can be combined into the best possible single tour.

<br><p> 
 ![eTSP_merge.jpg](eTSP_merge2.jpg)
<br><p> 

To do this, we can try all possible ways to merge each tour by rerouting across the cut and back and choose the least costly. This yields the following algorithm:


<p><span class="math display">\[\begin{array}{l}  
\mathit{eTSP}~(P) =  
\\  
~~~~\texttt{if}~|P|<2~\texttt{then}  
\\  
~~~~~~~~\texttt{raise}~\mathit{TooSmall}  
\\  
~~~~\texttt{else if}~|P| = 2~\texttt{then}  
\\  
~~~~~~~~\left\langle\, (P[0],P[1]),(P[1],P[0]) \,\right\rangle  
\\  
~~~~\texttt{else}  
\\  
~~~~~~~~\texttt{let}  
\\  
~~~~~~~~~~~~(P_\ell, P_r) = \mathit{split}~P~\texttt{along the longest dimension}  
\\  
~~~~~~~~~~~~(L, R) = (\mathit{eTSP}~P_\ell) \mid\mid{} (\mathit{eTSP}~P_r)  
\\  
~~~~~~~~~~~~(c,(e,e')) = \mathit{minVal}_{\mathit{first}} \left\{ (\mathit{swapCost}(e,e'),(e,e')) : e \in L, e' \in R \right\}  
\\  
~~~~~~~~\texttt{in}  
\\  
~~~~~~~~~~~~~~~~\mathit{swapEdges}~(\mathit{append}~(L,R),e,e')  
\\  
~~~~~~~~\texttt{end}  
\end{array}\]</span></p>

<p>The function $\mathit{minVal}_{\mathit{first}}$ uses the first value of the pairs to find the minimum, and returns the (first) pair with that minimum. The function $\mathit{swapEdges}(E,e,e')$ finds the edges $e$ and $e'$ and swaps the endpoints. As there are two ways to swap, it picks the cheaper one.</p>

   

**Correctness**: Does this algorithm compute a tour? Does this algorithm compute a minimum-cost tour?
    
We can show by induction that this algorithm always produces a tour. 

### However, the combine step does not necessarily produce a minimum cost tour!

<br>

Actually, we currently do not know of any  polynomial-work algorithm to solve this problem. In fact, the brute-force algorithm is essentially the best we can do. 


**Work/Span**:

This algorithm has two recursive calls that each operate on $n/2$ points. To combine the solution we must check $O(n^2)$ ways too cross the cut and compute the best. This requires $O(n^2)$ work and $O(\log n)$ span. 

So we have that the work is $W(n) = 2W(n/2) + O(n^2).$ This is a root-dominated recurrence, and thus $W(n) = O(n^2)$. 

The span is $S(n) = S(n/2) + O(\log n)$. This is a balanced recurrence with $\lg n$ levels, and so $S(n) = O(\log^2 n)$.
